In [1]:
import os
import pandas as pd
import numpy as np
import logging
from datetime import datetime, timedelta
import requests
import time
import json

from config import RECORDS_PATH
from machine_lib import login, set_alpha_properties
import concurrent.futures

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def get_submitted_alpha_ids(s, region, alpha_num):
    """
    获取已提交的alpha因子ID列表
    :param s: 会话对象
    :param region: 地区
    :param alpha_num: 需要获取的alpha数量
    :return: alpha ID列表
    """
    count = 0
    id_lst = []
    max_retries = 3
    retry_delay = 1  # 重试延迟时间（秒）
   
    for i in range(0, alpha_num, 100):
        retry_count = 0
        while retry_count < max_retries:
            try:
                url = f"https://api.worldquantbrain.com/users/self/alphas?limit=100&offset={i}" \
                      f"&status!=UNSUBMITTED" \
                      f"&settings.region={region}" \
                      f"&order=-is.sharpe"
               
                response = s.get(url)
                response.raise_for_status()  # 检查HTTP错误
               
                try:
                    alpha_list = response.json()["results"]
                    for alpha in alpha_list:
                        alpha_id = alpha["id"]
                        count += 1
                        id_lst.append(alpha_id)
                    break  # 成功获取数据，跳出重试循环
                   
                except (KeyError, json.JSONDecodeError) as e:
                    logger.error(f"解析响应数据失败: {str(e)}")
                    retry_count += 1
                    time.sleep(retry_delay)
                   
            except requests.exceptions.RequestException as e:
                logger.error(f"请求失败: {str(e)}")
                retry_count += 1
                time.sleep(retry_delay)
               
        if retry_count >= max_retries:
            logger.error(f"获取数据失败，已达到最大重试次数: offset={i}")
            break
           
    logger.info(f"成功获取 {count} 个alpha因子ID")
    return id_lst


class AutoCorrelationTester:
    def __init__(self, threshold=0.7, window_size=20, max_lag=100, cache_dir="cache"):
        """
        初始化自相关检测器
        :param threshold: 自相关系数阈值
        :param window_size: 滑动窗口大小
        :param max_lag: 最大滞后期
        :param cache_dir: 缓存目录
        """
        self.threshold = threshold
        self.window_size = window_size
        self.max_lag = max_lag
        self.session = login()
        self.cache_dir = cache_dir
        self.pnl_cache = {}
       
        # 确保缓存目录存在
        os.makedirs(cache_dir, exist_ok=True)
       
    def _get_cache_path(self, alpha_id):
        """获取缓存文件路径"""
        return os.path.join(self.cache_dir, f"{alpha_id}.pkl")
       
    def _load_from_cache(self, alpha_id):
        """从缓存加载数据"""
        cache_path = self._get_cache_path(alpha_id)
        if os.path.exists(cache_path):
            try:
                with open(cache_path, 'rb') as f:
                    return pd.read_pickle(f)
            except Exception as e:
                logger.error(f"从缓存加载数据失败: {alpha_id}, 错误: {str(e)}")
                return None
        return None
       
    def _save_to_cache(self, alpha_id, data):
        """保存数据到缓存"""
        cache_path = self._get_cache_path(alpha_id)
        try:
            with open(cache_path, 'wb') as f:
                pd.to_pickle(data, f)
        except Exception as e:
            logger.error(f"保存数据到缓存失败: {alpha_id}, 错误: {str(e)}")
           
    def get_alpha_pnl(self, alpha_id):
        """
        获取alpha的PNL数据，优先从缓存读取
        :param alpha_id: alpha的ID
        :return: PNL数据DataFrame
        """
        # 检查内存缓存
        if alpha_id in self.pnl_cache:
            return self.pnl_cache[alpha_id]
           
        # 检查文件缓存
        cached_data = self._load_from_cache(alpha_id)
        if cached_data is not None:
            self.pnl_cache[alpha_id] = cached_data
            return cached_data
           
        # 从API获取数据
        try:
            # 获取PNL数据
            url = f"https://api.worldquantbrain.com/alphas/{alpha_id}/recordsets/pnl"
           
            # 循环获取数据，直到没有Retry-After头
            while True:
                pnl_response = self.session.get(url)
                retry_after = pnl_response.headers.get('Retry-After')
                if not retry_after:
                    break
                logger.info(f'需要等待 {retry_after} 秒后重试')
                time.sleep(float(retry_after))
               
            if pnl_response.status_code != 200:
                logger.error(f"获取PNL数据失败: {alpha_id}, 状态码: {pnl_response.status_code}")
                return None
               
            # 检查响应内容
            response_text = pnl_response.text
            if not response_text:
                logger.error(f"获取PNL数据失败: {alpha_id}, 响应内容为空")
                return None
               
            # 尝试解析JSON
            try:
                pnl_data = json.loads(response_text)
            except json.JSONDecodeError as e:
                logger.error(f"解析PNL数据失败: {alpha_id}, 错误: {str(e)}")
                return None
               
            if not pnl_data.get("records"):
                logger.warning(f"没有PNL数据: {alpha_id}")
                return None
               
            # 转换为DataFrame
            try:
                dates = [record[0] for record in pnl_data["records"]]
                values = [record[1] for record in pnl_data["records"]]
                pnl_df = pd.DataFrame({"date": dates, "pnl": values})
                pnl_df["date"] = pd.to_datetime(pnl_df["date"])
                pnl_df.set_index("date", inplace=True)
               
                # 保存到缓存
                self.pnl_cache[alpha_id] = pnl_df
                self._save_to_cache(alpha_id, pnl_df)
               
                logger.info(f"成功获取 {alpha_id} 的PNL数据")
                return pnl_df
               
            except Exception as e:
                logger.error(f"处理PNL数据失败: {alpha_id}, 错误: {str(e)}")
                return None
               
        except Exception as e:
            logger.error(f"获取alpha数据时发生错误: {alpha_id}, 错误: {str(e)}")
            return None

    def calculate_returns(self, pnl_series):
        """
        计算收益率序列
        :param pnl_series: PNL时间序列
        :return: 收益率序列
        """
        return pnl_series.diff()

    def calculate_pearson_autocorrelation(self, returns, lag):
        """
        计算皮尔逊自相关系数
        :param returns: 收益率序列
        :param lag: 滞后期
        :return: 皮尔逊相关系数
        """
        if len(returns) <= lag:
            return 0.0
       
        # 计算滞后期序列
        lagged_returns = returns.shift(lag)
       
        # 删除NaN值
        valid_returns = returns[lag:]
        valid_lagged = lagged_returns[lag:]
       
        # 确保没有NaN值
        mask = ~(np.isnan(valid_returns) | np.isnan(valid_lagged))
        valid_returns = valid_returns[mask]
        valid_lagged = valid_lagged[mask]
       
        if len(valid_returns) < 2:
            return 0.0
           
        # 使用numpy计算皮尔逊相关系数
        correlation = np.corrcoef(valid_returns, valid_lagged)[0, 1]
        return correlation

    def calculate_local_self_correlation(self, pnl_series):
        """
        计算本地自相关系数
        :param pnl_series: PNL时间序列
        :return: 最大自相关系数
        """
        if len(pnl_series) < self.window_size:
            return 0.0
           
        # 计算收益率
        returns = self.calculate_returns(pnl_series)
       
        # 使用滑动窗口计算自相关系数
        max_corr = 0
        for lag in range(1, self.window_size):
            corr = self.calculate_pearson_autocorrelation(returns, lag)
            if abs(corr) > abs(max_corr):
                max_corr = corr
               
        return max_corr
   

    def calculate_cross_correlation(self, returns1, returns2):
        """
        计算两个时间序列之间的互相关系数
        :param returns1: 第一个alpha的收益率序列
        :param returns2: 第二个alpha的收益率序列
        :return: 互相关系数及相关统计信息
        """
        # 确保两个序列长度一致
        min_length = min(len(returns1), len(returns2))
        returns1 = returns1[:min_length]
        returns2 = returns2[:min_length]
       
        # 删除NaN值
        mask = ~(np.isnan(returns1) | np.isnan(returns2))
        returns1 = returns1[mask]
        returns2 = returns2[mask]
       
        if len(returns1) < 2:
            return {
                "correlation": 0.0,
                "statistics": {
                    "length": len(returns1),
                    "returns1_mean": 0.0,
                    "returns1_std": 0.0,
                    "returns2_mean": 0.0,
                    "returns2_std": 0.0,
                    "covariance": 0.0
                }
            }
           
        # 计算基本统计量
        stats = {
            "length": len(returns1),
            "returns1_mean": returns1.mean(),
            "returns1_std": returns1.std(),
            "returns2_mean": returns2.mean(),
            "returns2_std": returns2.std(),
            "covariance": np.cov(returns1, returns2)[0, 1]
        }
       
        # 标准化数据
        returns1_std = (returns1 - stats["returns1_mean"]) / stats["returns1_std"]
        returns2_std = (returns2 - stats["returns2_mean"]) / stats["returns2_std"]
       
        # 计算相关系数
        correlation = np.corrcoef(returns1_std, returns2_std)[0, 1]
        correlation = round(correlation, 4)
       
        return {
            "correlation": correlation,
            "statistics": stats
        }

    def calculate_lag_correlation(self, returns1, returns2):
        """
        计算returns2相对于returns1的滞后相关系数
        :param returns1: 基准alpha的收益率序列
        :param returns2: 滞后alpha的收益率序列
        :return: 滞后相关系数列表
        """
        correlations = []
       
        # 确保两个序列长度一致
        min_length = min(len(returns1), len(returns2))
        returns1 = returns1[:min_length]
        returns2 = returns2[:min_length]
       
        for lag in range(1, self.max_lag + 1):
            # returns2滞后于returns1
            r1 = returns1[:-lag]
            r2 = returns2[lag:len(r1)+lag]
           
            # 确保没有NaN值
            mask = ~(np.isnan(r1) | np.isnan(r2))
            r1 = r1[mask]
            r2 = r2[mask]
           
            if len(r1) < 2:
                corr = 0.0
            else:
                # 确保两个序列长度一致
                min_len = min(len(r1), len(r2))
                r1 = r1[:min_len]
                r2 = r2[:min_len]
                corr = np.corrcoef(r1, r2)[0, 1]
               
            correlations.append((lag, corr))
        return correlations

    def check_autocorrelation(self, pnl_data):
        """
        检查自相关性
        :param pnl_data: PNL数据，可以是DataFrame或Series
        :return: 检测结果字典
        """
        try:
            # 确保输入数据格式正确
            if isinstance(pnl_data, pd.DataFrame):
                if 'pnl' in pnl_data.columns:
                    pnl_series = pnl_data['pnl']
                else:
                    raise ValueError("DataFrame必须包含'pnl'列")
            elif isinstance(pnl_data, pd.Series):
                pnl_series = pnl_data
            else:
                raise ValueError("输入数据必须是DataFrame或Series")

            # 计算最大自相关系数
            max_corr = self.calculate_local_self_correlation(pnl_series)
           
            # 生成检测结果
            result = {
                "test": "AUTOCORRELATION",
                "result": "PASS" if abs(max_corr) < self.threshold else "FAIL",
                "threshold": self.threshold,
                "max_correlation": max_corr,
                "window_size": self.window_size
            }
           
            return result
           
        except Exception as e:
            logger.error(f"自相关检测失败: {str(e)}")
            return {
                "test": "AUTOCORRELATION",
                "result": "ERROR",
                "error": str(e)
            }

    def batch_check_autocorrelation(self, alpha_ids, max_workers=4):
        """
        批量检查多个alpha的自相关性
        :param alpha_ids: alpha ID列表
        :param max_workers: 最大工作线程数
        :return: 检测结果列表
        """
        results = []
       
        # 创建线程池
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # 提交任务
            future_to_alpha = {
                executor.submit(self.check_autocorrelation, alpha_id): alpha_id
                for alpha_id in alpha_ids
            }
           
            # 处理完成的任务
            for future in concurrent.futures.as_completed(future_to_alpha):
                alpha_id = future_to_alpha[future]
                try:
                    result = future.result()
                    results.append(result)
                    logger.info(f"完成检查 {alpha_id}")
                except Exception as e:
                    logger.error(f"检查 {alpha_id} 时发生错误: {str(e)}")
                   
        return results
       
    def batch_calculate_cross_correlation(self, alpha_pairs, max_workers=4):
        """
        批量计算多个alpha对之间的互相关系数
        :param alpha_pairs: alpha对列表，每个元素为(alpha_id1, alpha_id2)
        :param max_workers: 最大工作线程数
        :return: 相关系数结果列表
        """
        results = []
       
        # 创建线程池
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # 提交任务
            future_to_pair = {
                executor.submit(self._calculate_pair_correlation, pair): pair
                for pair in alpha_pairs
            }
           
            # 处理完成的任务
            for future in concurrent.futures.as_completed(future_to_pair):
                pair = future_to_pair[future]
                try:
                    result = future.result()
                    results.append({
                        "alpha1": pair[0],
                        "alpha2": pair[1],
                        "result": result
                    })
                except Exception as e:
                    logger.error(f"计算 {pair[0]} 和 {pair[1]} 的相关系数时发生错误: {str(e)}")
                   
        return results
       
    def _calculate_pair_correlation(self, pair):
        """
        计算一对alpha的相关系数
        :param pair: (alpha_id1, alpha_id2)
        :return: 相关系数结果
        """
        alpha_id1, alpha_id2 = pair
       
        # 获取PNL数据
        pnl1 = self.get_alpha_pnl(alpha_id1)
        pnl2 = self.get_alpha_pnl(alpha_id2)
       
        if pnl1 is None or pnl2 is None:
            return None
           
        # 计算收益率
        returns1 = self.calculate_returns(pnl1["pnl"])
        returns2 = self.calculate_returns(pnl2["pnl"])
       
        # 计算相关系数
        return self.calculate_cross_correlation(returns1, returns2)

class AlphaCorrelationTester:
    def __init__(self, threshold=0.7):
        self.threshold = threshold
        self.session = login()
        self.base_url = "https://api.worldquantbrain.com"
        self.pnl_dir = "data/pnl"
        self.cache_dir = "cache"
       
        # 确保目录存在
        os.makedirs(self.pnl_dir, exist_ok=True)
        os.makedirs(self.cache_dir, exist_ok=True)
       
        # 初始化缓存
        self.pnl_cache = {}
       
    def get_unsubmitted_alphas(self, region, limit=100, start_date=None, end_date=None, sharpe_th=1.25, fitness_th=1.0, longCount_th=10, shortCount_th=10,color_exclude = ''):
        print("开始获取未提交的alpha列表...")
        unsubmitted_alphas = []
        offset = 0
   
    # 设置默认日期范围
        if start_date is None:
            start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
        if end_date is None:
            end_date = datetime.now().strftime("%Y-%m-%d")
       
        while True:
            try:
                url = f"{self.base_url}/users/self/alphas" \
                    f"?limit={limit}&offset={offset}" \
                    "&status=UNSUBMITTED" \
                    f"&settings.region={region}" \
                    f"&dateCreated%3E={start_date}T00:00:00-04:00" \
                    f"&dateCreated%3C={end_date}T00:00:00-04:00" \
                    f"&is.sharpe%3E={sharpe_th}" \
                    f"&is.fitness%3E={fitness_th}" \
                    f"&is.longCount%3E={longCount_th}" \
                    f"&is.shortCount%3E={shortCount_th}" \
                    f"&color!={color_exclude}"\
                    "&order=-is.sharpe"
                 
                print(f"正在获取未提交alpha列表，offset={offset}...")
                response = self.session.get(url, timeout=30)  # 添加超时
                if response.status_code != 200:
                    print(f"获取未提交alpha失败: {response.status_code}")
                    break
               
                try:
                    alpha_list = response.json()["results"]
                    if not alpha_list:
                        break
                   
                    for alpha_detail in alpha_list:
                    # 提取所有alpha详细信息
                        alpha_info = {
                        "id": alpha_detail["id"],
                        "type": alpha_detail["type"],
                        "author": alpha_detail["author"],
                        "instrumentType": alpha_detail["settings"]["instrumentType"],
                        "region": alpha_detail["settings"]["region"],
                        "universe": alpha_detail["settings"]["universe"],
                        "delay": alpha_detail["settings"]["delay"],
                        "decay": alpha_detail["settings"]["decay"],
                        "neutralization": alpha_detail["settings"]["neutralization"],
                        "truncation": alpha_detail["settings"]["truncation"],
                        "pasteurization": alpha_detail["settings"]["pasteurization"],
                        "unitHandling": alpha_detail["settings"]["unitHandling"],
                        "nanHandling": alpha_detail["settings"]["nanHandling"],
                        "language": alpha_detail["settings"]["language"],
                        "visualization": alpha_detail["settings"]["visualization"],
                        "code": alpha_detail["regular"]["code"],
                        "description": alpha_detail["regular"]["description"],
                        "operatorCount": alpha_detail["regular"]["operatorCount"],
                        "dateCreated": alpha_detail["dateCreated"],
                        "dateSubmitted": alpha_detail["dateSubmitted"],
                        "dateModified": alpha_detail["dateModified"],
                        "name": alpha_detail["name"],
                        "favorite": alpha_detail["favorite"],
                        "hidden": alpha_detail["hidden"],
                        "color": alpha_detail["color"],
                        "category": alpha_detail["category"],
                        "tags": alpha_detail["tags"],
                        "classifications": alpha_detail["classifications"],
                        "grade": alpha_detail["grade"],
                        "stage": alpha_detail["stage"],
                        "status": alpha_detail["status"],
                        "pnl": alpha_detail["is"]["pnl"],
                        "bookSize": alpha_detail["is"]["bookSize"],
                        "longCount": alpha_detail["is"]["longCount"],
                        "shortCount": alpha_detail["is"]["shortCount"],
                        "turnover": alpha_detail["is"]["turnover"],
                        "returns": alpha_detail["is"]["returns"],
                        "drawdown": alpha_detail["is"]["drawdown"],
                        "margin": alpha_detail["is"]["margin"],
                        "fitness": alpha_detail["is"]["fitness"],
                        "sharpe": alpha_detail["is"]["sharpe"],
                        "startDate": alpha_detail["is"]["startDate"],
                        "checks": alpha_detail["is"]["checks"],
                        "os": alpha_detail["os"],
                        "train": alpha_detail["train"],
                        "test": alpha_detail["test"],
                        "prod": alpha_detail["prod"],
                        "competitions": alpha_detail["competitions"],
                        "themes": alpha_detail["themes"],
                        "team": alpha_detail["team"]
                    }
                   
                    # 检查checks结果
                        checks_df = pd.DataFrame(alpha_detail["is"]["checks"])
                        pyramids = next(
                            ([y['name'] for y in item['pyramids']] for item in alpha_detail["is"]["checks"] if item['name'] == 'MATCHES_PYRAMID'), None)
                   
                    # 如果任何检查结果为FAIL，设置颜色为红色
                        if any(checks_df["result"] == "FAIL"):
                            set_alpha_properties(self.session, alpha_detail["id"], color='RED')
                            continue
                       
                        # 添加到未提交alpha列表
                        unsubmitted_alphas.append({
                            "id": alpha_detail["id"],
                            "sharpe": alpha_detail["is"]["sharpe"],
                            "fitness": alpha_detail["is"]["fitness"],
                            "margin": alpha_detail["is"]["margin"],
                            "full_info": alpha_info  # 包含所有详细信息
                        })
                   
                    print(f"已获取 {len(unsubmitted_alphas)} 个未提交alpha")
                    offset += limit
                except Exception as e:
                    print(f"解析未提交alpha数据失败: {str(e)}")
                    break
               
            except requests.exceptions.Timeout:
                print("请求超时，正在重试...")
                continue
            except Exception as e:
                print(f"获取未提交alpha时发生错误: {str(e)}")
                break
           
        print(f"获取未提交alpha完成，共 {len(unsubmitted_alphas)} 个")
        return unsubmitted_alphas
       
    def get_submitted_alphas(self, region, limit=100):

        """获取已提交的alpha列表，并下载所有PNL数据"""
        print("开始获取已提交的alpha列表...")
        submitted_alphas = []
       
        downloaded_ids = set()
       
        # 首先从本地文件夹读取已下载的alpha
        if os.path.exists(self.pnl_dir):
            print("从本地文件夹读取已下载的alpha...")
            for filename in os.listdir(self.pnl_dir):
                if filename.endswith(".csv"):
                    alpha_id = filename[:-4]  # 去掉.csv后缀
                    downloaded_ids.add(alpha_id)
                    submitted_alphas.append({
                        "id": alpha_id,
                        "sharpe": 0.0,  # 本地文件没有这些信息
                        "fitness": 0.0
                    })
            print(f"从本地读取到 {len(submitted_alphas)} 个已下载alpha")
       
        # 从API获取所有已提交alpha
        print("从API获取所有已提交alpha...")
        offset = 0
        while True:
            try:
                url = f"{self.base_url}/users/self/alphas" \
                      f"?limit={limit}&offset={offset}" \
                      "&status!=UNSUBMITTED" \
                      f"&settings.region={region}" \
                      "&order=-is.sharpe"
                     
                print(f"正在获取已提交alpha列表，offset={offset}...")
                response = self.session.get(url, timeout=30)
                if response.status_code != 200:
                    print(f"获取已提交alpha失败: {response.status_code}")
                    break
                   
                try:
                    alpha_list = response.json()["results"]
                    if not alpha_list:
                        break
                       
                    for alpha in alpha_list:
                        alpha_id = alpha["id"]
                        # 如果还没有下载过，下载PNL数据
                        if alpha_id not in downloaded_ids:
                            print(f"下载alpha {alpha_id} 的PNL数据...")
                            self._download_alpha_pnl(alpha_id)
                            downloaded_ids.add(alpha_id)
                           
                        # 添加到列表
                        if not any(a["id"] == alpha_id for a in submitted_alphas):
                            submitted_alphas.append({
                                "id": alpha_id,
                                "sharpe": alpha["is"]["sharpe"],
                                "fitness": alpha["is"]["fitness"]
                            })
                           
                    print(f"已获取 {len(submitted_alphas)} 个已提交alpha")
                    offset += limit
                except Exception as e:
                    print(f"解析已提交alpha数据失败: {str(e)}")

                    break
                   
            except requests.exceptions.Timeout:
                print("请求超时，正在重试...")
                continue
            except Exception as e:
                print(f"获取已提交alpha时发生错误: {str(e)}")
                break
               
        print(f"获取已提交alpha完成，共 {len(submitted_alphas)} 个")
        return submitted_alphas
       
    def _download_alpha_pnl(self, alpha_id):

        """下载alpha的PNL数据到本地"""
        try:
            # 确保目录存在
            os.makedirs(self.pnl_dir, exist_ok=True)
           
            # 构建PNL数据URL
            url = f"{self.base_url}/alphas/{alpha_id}/recordsets/pnl"
           
            # 获取PNL数据
            response = self.session.get(url, timeout=30)
            if response.status_code != 200:
                print(f"获取alpha {alpha_id} 的PNL数据失败: {response.status_code}")
                return
               
            # 解析PNL数据
            pnl_data = response.json()
            if not pnl_data.get("records"):
                print(f"alpha {alpha_id} 没有PNL数据")
                return
               
            # 转换为DataFrame
            df = pd.DataFrame(pnl_data["records"], columns=["date", "pnl"])
            df["date"] = pd.to_datetime(df["date"])
            df.set_index("date", inplace=True)
           
            # 保存到本地
            file_path = os.path.join(self.pnl_dir, f"{alpha_id}.csv")
            df.to_csv(file_path)
            print(f"已保存alpha {alpha_id} 的PNL数据到 {file_path}")
           
        except Exception as e:
            print(f"下载alpha {alpha_id} 的PNL数据失败: {str(e)}")

    def _get_performance(self, alpha_id):
        MAX_RETRIES = 30  # 最大重试次数
        RETRY_DELAY = 5  # 重试间隔时间（秒）
        """下载 alpha 的 PNL 数据到本地"""
        retries = 0
        while retries < MAX_RETRIES:
            try:
            # 构建 PNL 数据 URL
               # url = f"{self.base_url}/competitions/IQC2025S1/alphas/{alpha_id}/before-and-after-performance"
               #修改处 
                url = f" {self.base_url}/alphas/{alpha_id}/recordsets/pnl"
            # 获取 PNL 数据
                response = self.session.get(url, timeout=30)

                # 检查 API 响应状态码

                if response.status_code != 200:
                    print(f"获取 alpha {alpha_id} 的 PNL 数据失败，状态码: {response.status_code}，第 {retries + 1} 次重试...")
                    retries += 1
                    time.sleep(RETRY_DELAY)
                    continue

            # 解析 PNL 数据
                performance_score_res = response.json()

            # 检查 score 数据是否存在
                if not performance_score_res.get("score"):
                    print(f"alpha {alpha_id} 没有 score 数据")
                    return

                performance_score_before = performance_score_res.get("score")['before']
                performance_score_after = performance_score_res.get("score")['after']
                performance_score = performance_score_after - performance_score_before
                return performance_score

            except Exception as e:
                print(f"获取 alpha {alpha_id} 的 score 数据失败: {str(e)}，第 {retries + 1} 次重试...")
                retries += 1
                time.sleep(RETRY_DELAY)

        print(f"达到最大重试次数，无法获取 alpha {alpha_id} 的 PNL 数据。")
        return None

    def check_all_unsubmitted_correlations(self, region="USA", max_workers=4, start_date=None, end_date=None, sharpe_th=1.25, fitness_th=1.0,longCount_th=10, shortCount_th=10, color_exclude='RED'):
        """检查所有未提交alpha与已提交alpha的互相关系数"""
        print("开始检查所有未提交alpha的相关系数...")
       
        # 获取未提交alpha列表
        unsubmitted = self.get_unsubmitted_alphas(region, start_date=start_date, end_date=end_date, sharpe_th=sharpe_th, fitness_th=fitness_th,longCount_th=longCount_th, shortCount_th=shortCount_th,color_exclude=color_exclude)
        if not unsubmitted:
            print("获取未提交alpha列表失败")
            return
           
        print(f"获取到 {len(unsubmitted)} 个未提交alpha")
       
        # 获取已提交alpha列表
        submitted = self.get_submitted_alphas(region)
        if not submitted:
            print("获取已提交alpha列表失败")
            return
           
        print(f"获取到 {len(submitted)} 个已提交alpha")
       
        # 创建AutoCorrelationTester实例
        tester = AutoCorrelationTester(threshold=self.threshold, cache_dir=self.cache_dir)
       
        # 预先获取所有已提交alpha的PNL数据
        print("\n预先加载已提交alpha的PNL数据...")
        for i, sub in enumerate(submitted, 1):
            sub_id = sub["id"]
            print(f"\r加载进度: {i}/{len(submitted)} ({i/len(submitted)*100:.1f}%)", end="")
            tester.get_alpha_pnl(sub_id)
        print("\n已提交alpha的PNL数据加载完成！")
       
        # 存储所有未提交alpha的最大相关系数信息
        all_results = {}
        low_correlation_alphas = set()
        high_correlation_alphas = set()
        # 处理每个未提交alpha
        total = len(unsubmitted)
        for i, unsub in enumerate(unsubmitted, 1):
            unsub_id = unsub["id"]
            unsub_sharpe = unsub["sharpe"]
            unsub_fitness = unsub["fitness"]
            unsub_margin = unsub["margin"]
            print(f"\r处理进度: {i}/{total} ({i/total*100:.1f}%) - 当前处理: {unsub_id}", end="")
           
            # 先获取未提交alpha的PNL数据
            unsub_pnl = tester.get_alpha_pnl(unsub_id)
            performance = self._get_performance(unsub_id)
            if unsub_pnl is None:
                print(f"\n跳过 {unsub_id}: 无法获取PNL数据")
                continue
               
            # 创建alpha对并计算相关系数
            alpha_pairs = [(unsub_id, sub["id"]) for sub in submitted]
            results = tester.batch_calculate_cross_correlation(alpha_pairs, max_workers=max_workers)
           
            # 处理结果
            max_correlation = 0.0
            max_correlation_submitted_id = None
           
            for result in results:
                if result["result"] is None:
                    continue
                correlation = result["result"]["correlation"]
                submitted_id = result["alpha2"]
               
                if abs(correlation) > abs(max_correlation):
                    max_correlation = correlation
                    max_correlation_submitted_id = submitted_id
           
            # 保存结果
            all_results[unsub_id] = {
                "max_correlation": max_correlation,
                "submitted_id": max_correlation_submitted_id,
                "unsub_sharpe": unsub_sharpe,
                "unsub_fitness": unsub_fitness,
                "unsub_margin": unsub_margin,
                "performance" : performance
            }
            if abs(max_correlation) >= self.threshold:
                high_correlation_alphas.add(unsub_id)
            # 如果最大相关系数小于阈值，添加到需要设置颜色的集合
            if abs(max_correlation) < self.threshold:
                low_correlation_alphas.add(unsub_id)
               
        print("\n所有alpha处理完成！")
       
        # 保存所有结果到文件
        self._save_all_results(all_results)
        if high_correlation_alphas:
            print(f"\n设置 {len(high_correlation_alphas)} 个高相关性alpha的颜色为红色...")
            self._set_high_correlation_color(high_correlation_alphas)
        else:

            print("\n没有需要设置颜色的alpha")
        # 统一设置颜色
        if low_correlation_alphas:
            print(f"\n设置 {len(low_correlation_alphas)} 个低相关性alpha的颜色为紫色...")
            self._set_low_correlation_color(low_correlation_alphas)
        else:
            print("\n没有需要设置颜色的alpha")
               
        print("\n处理完成！请查看 records/check_results.txt 文件获取详细信息。")
       
    def _save_all_results(self, results):
        """保存所有未提交alpha的最大相关系数信息到文件"""
        # 确保目录存在
        os.makedirs("records", exist_ok=True)
       
        # 按相关系数从大到小排序
        sorted_results = sorted(
            results.items(),
            key=lambda x: abs(x[1]["max_correlation"]),
            reverse=True
        )
       
        # 写入文件
        with open("records/check_results.csv", "w", encoding="utf-8") as f:
            f.write("未提交alpha_id,已提交alpha_id,最大相关系数,sharpe,fitness,margin,performance\n")
            for unsub_id, info in sorted_results:
                f.write(f"{unsub_id},{info['submitted_id']},{info['max_correlation']:.4f},{info['unsub_sharpe']:.4f},{info['unsub_fitness']:.4f},{info['unsub_margin']:.4f},{info['performance']}\n")
               
        print(f"已将结果保存到 records/check_results.csv")

    def _set_low_correlation_color(self, alpha_ids):
        """将低相关性的alpha设置为紫色"""
        if not alpha_ids:
            return
           
        success_count = 0
        fail_count = 0
       
        for alpha_id in alpha_ids:
            try:
                # 使用set_alpha_properties设置颜色
                set_alpha_properties(self.session, alpha_id, color="PURPLE")
                success_count += 1
                time.sleep(0.1)  # 避免请求过于频繁
               
            except Exception as e:
                logger.error(f"设置alpha {alpha_id} 颜色失败: {str(e)}")
                fail_count += 1
               
        logger.info(f"颜色设置: 成功 {success_count} 个, 失败 {fail_count} 个")
   
    def _set_high_correlation_color(self, alpha_ids):
        """将高相关性的alpha设置为红色"""
        if not alpha_ids:
            return
           
        success_count = 0
        fail_count = 0
       
        for alpha_id in alpha_ids:
            try:
                # 使用set_alpha_properties设置颜色
                set_alpha_properties(self.session, alpha_id, color="RED")
                success_count += 1
                time.sleep(0.1)  # 避免请求过于频繁
               
            except Exception as e:
                logger.error(f"设置alpha {alpha_id} 颜色失败: {str(e)}")
                fail_count += 1
               
        logger.info(f"颜色设置: 成功 {success_count} 个, 失败 {fail_count} 个")

def main():
    # 创建测试器实例
    tester = AlphaCorrelationTester(threshold=0.7)
   
    # 设置日期范围（可选）
    start_date = "2025-09-30"  # 默认30天前
    end_date = "2025-10-31"  # 默认今天
    sharpe_th = 1.25
    fitness_th = 1
    #longCount_th=10
    #shortCount_th=10
    color_exclude="RED"
   
    # 检查所有未提交alpha的相关系数
    tester.check_all_unsubmitted_correlations(
        region="USA",
        max_workers=4,



        start_date=start_date,
        end_date=end_date,
        sharpe_th=sharpe_th,
        fitness_th=fitness_th,
        #longCount_th=longCount_th,
        #shortCount_th=shortCount_th,
        color_exclude=color_exclude
    )
       
if __name__ == "__main__":
    main()

b'{"user":{"id":"ZY20347"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
开始检查所有未提交alpha的相关系数...
开始获取未提交的alpha列表...
正在获取未提交alpha列表，offset=0...
已获取 68 个未提交alpha
正在获取未提交alpha列表，offset=100...
获取未提交alpha完成，共 68 个
获取到 68 个未提交alpha
开始获取已提交的alpha列表...
从本地文件夹读取已下载的alpha...
从本地读取到 64 个已下载alpha
从API获取所有已提交alpha...
正在获取已提交alpha列表，offset=0...
下载alpha vRKgblKz 的PNL数据...
下载alpha vRKgblKz 的PNL数据失败: Expecting value: line 1 column 1 (char 0)
下载alpha 6XmJM9np 的PNL数据...
下载alpha 6XmJM9np 的PNL数据失败: Expecting value: line 1 column 1 (char 0)
下载alpha blGA39rM 的PNL数据...
下载alpha blGA39rM 的PNL数据失败: Expecting value: line 1 column 1 (char 0)
下载alpha 9qZ7mpao 的PNL数据...
下载alpha 9qZ7mpao 的PNL数据失败: Expecting value: line 1 column 1 (char 0)
下载alpha 1YqeoMn6 的PNL数据...
下载alpha 1YqeoMn6 的PNL数据失败: Expecting value: line 1 column 1 (char 0)
下载alpha E5OnA6dP 的PNL数据...
下载alpha E5OnA6dP 的PNL数据失败: Expecting value: line 1 column 1 (char 0)
下载alpha kqxlPNlK 的PNL数据...
下载alpha kqxlPNlK 的PNL数据失败: Expecting value: l

2025-10-28 15:25:43,609 - INFO - 需要等待 1.0 秒后重试
2025-10-28 15:25:44,917 - INFO - 成功获取 pwmXeVv3 的PNL数据


alpha pwmXeVv3 没有 score 数据
处理进度: 2/68 (2.9%) - 当前处理: ZY65NOZd

2025-10-28 15:25:46,373 - INFO - 需要等待 1.0 秒后重试
2025-10-28 15:25:47,670 - INFO - 成功获取 ZY65NOZd 的PNL数据


alpha ZY65NOZd 没有 score 数据
处理进度: 3/68 (4.4%) - 当前处理: vR8Mkvwz

...

alpha 6X7VwZ87 没有 score 数据
处理进度: 68/68 (100.0%) - 当前处理: ak6ZA9G5

2025-10-28 15:28:33,829 - INFO - 需要等待 1.0 秒后重试
2025-10-28 15:28:35,575 - INFO - 成功获取 ak6ZA9G5 的PNL数据


alpha ak6ZA9G5 没有 score 数据

所有alpha处理完成！
已将结果保存到 records/check_results.csv

设置 68 个高相关性alpha的颜色为红色...


2025-10-28 15:29:12,330 - INFO - 颜色设置: 成功 68 个, 失败 0 个



没有需要设置颜色的alpha

处理完成！请查看 records/check_results.txt 文件获取详细信息。


In [2]:
user阶段注释掉了多空持仓的条件

NameError: name 'user阶段注释掉了多空持仓的条件' is not defined